In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt

In [26]:
# 设定随机种子以确保结果可复现
torch.manual_seed(42)

# 超参数范围
# hidden_layers = [2, 3, 4]
hidden_layers = [4]
# learning_rates = [0.01, 0.0001, 0.00001]
learning_rates = [0.0001]
# epochs_range = range(1, 11)
epochs_range = [9]
batch_sizes = [16, 32, 64, 128]
# batch_sizes = [ 64]

# 用于存储结果
results = {}
best_accuracy = 0
best_params = None

In [9]:
# 加载 MNIST 数据集
transform = transforms.Compose([
    transforms.RandomRotation(degrees=20),            # 随机旋转图像，范围是 [-20度, 20度]
    transforms.RandomHorizontalFlip(p=0.5),           # 50%的概率水平翻转图像
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # 随机平移，最大平移10%
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))              # 标准化
])

# train_dataset = datasets.MNIST(root='/public/group_data_2023/luohh/Class/01.OmicsAndAI/01.Materials/homework/data', train=True, 
#                                            transform=transform, download=True)
# test_dataset = datasets.MNIST(root='/public/group_data_2023/luohh/Class/01.OmicsAndAI/01.Materials/homework/data', train=False, 
#                                           transform=transform, download=True)

dataset = datasets.MNIST(root='/public/group_data_2023/luohh/Class/01.OmicsAndAI/01.Materials/homework/data', train=True, download=True, 
                         transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, validation_dataset = random_split(dataset, [train_size, test_size])
test_dataset = datasets.MNIST(root='/public/group_data_2023/luohh/Class/01.OmicsAndAI/01.Materials/homework/data', train=False, download=True, 
                              transform=transform)

In [20]:
# 定义 MLP 模型
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers):
        super(MLP, self).__init__()
        layers = []
        layers.append(nn.Flatten())
        layers.append(nn.Linear(input_size, 128))  # 输入层

        for _ in range(hidden_layers - 1):  # 添加指定数量的隐藏层
            layers.append(nn.ReLU())
            layers.append(nn.Linear(128, 128))  # 隐藏层

        layers.append(nn.ReLU())
        layers.append(nn.Linear(128, 10))  # 输出层
        layers.append(nn.LogSoftmax(dim=1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [25]:
# 遍历超参数
for hidden_layer in hidden_layers:
    for lr in learning_rates:
        for epochs in epochs_range:
            for batch_size in batch_sizes:
                # 创建数据加载器
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

                model = MLP(28 * 28, hidden_layer)  # 输入特征为 28 * 28
                criterion = nn.NLLLoss()
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # 训练模型
                model.train()
                for epoch in range(epochs):
                    for images, labels in train_loader:
                        optimizer.zero_grad()
                        output = model(images)
                        loss = criterion(output, labels)
                        loss.backward()
                        optimizer.step()

                # 评估模型
                model.eval()
                y_true, y_pred = [], []
                with torch.no_grad():
                    for images, labels in validation_loader:
                        output = model(images)
                        preds = torch.argmax(output, dim=1)
                        y_true.extend(labels.numpy())
                        y_pred.extend(preds.numpy())

                accuracy = accuracy_score(y_true, y_pred)

                # 记录结果
                params = (hidden_layer, lr, epochs, batch_size)
                results[params] = accuracy
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = params

                print(f"Parameters: {params}, Accuracy: {accuracy}")

Parameters: (4, 0.0001, 5, 16), Accuracy: 0.8203333333333334
Parameters: (4, 0.0001, 5, 32), Accuracy: 0.786
Parameters: (4, 0.0001, 5, 64), Accuracy: 0.7715
Parameters: (4, 0.0001, 5, 128), Accuracy: 0.7255
